In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import glob
import random
from google.colab import drive
from PIL import Image
from torch.nn import functional as F

def gen_split(root_dir, stackSize, train, frameType):
    Dataset = []
    Labels = []
    NumFrames = []
    if train==True:
      list_folder=['S1']
    elif train==False:
      list_folder=['S2']
    for dir_user in list_folder: #S1,S2,S3,S4
      class_id=0
      dir = os.path.join(root_dir, dir_user)
      for target in sorted(os.listdir(dir)): #action1, action2
        dir1 = os.path.join(dir, target)
        insts = sorted(os.listdir(dir1))
        if insts != []:
          i=0
          for inst in insts:
            if frameType=='r':
              inst_dir = os.path.join(dir1, inst, 'rgb')
            elif frameType=='m':
              inst_dir = os.path.join(dir1, inst, 'mmaps')
            numFrames = len(glob.glob1(inst_dir, '*.png'))
            if numFrames >= stackSize or inst_dir=='/content/gdrive/My Drive/Project FPAR2/ego-rnn/GTEA61/processed_frames2/S3/stir_spoon,cup/2/mmaps' or inst_dir=='/content/gdrive/My Drive/Project FPAR2/ego-rnn/GTEA61/processed_frames2/S4/pour_sugar,spoon,cup/1/mmaps':
              Dataset.append(inst_dir)
              Labels.append(class_id)
              NumFrames.append(numFrames)
            else:
              i+=1
          if len(insts)==i:
            print(f'This should not be considered! Frametype: {frametype}')
            print(dir1)
        class_id+=1
    return Dataset, Labels, NumFrames   

class makeDataset(Dataset):

    def __init__(self, root_dir, spatial_transform=None, seqLen=20,
                 train=True, mulSeg=False, numSeg=1, fmt='.png', frameType='r'):

        self.frameType=frameType
        self.root_dir=root_dir
        self.images, self.labels, self.numFrames = gen_split(root_dir, 5, train, frameType)
        self.spatial_transform = spatial_transform
        self.train = train
        self.mulSeg = mulSeg
        self.numSeg = numSeg
        self.seqLen = seqLen
        self.fmt = fmt

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        vid_name = self.images[idx]
        label = self.labels[idx]
        numFrame = self.numFrames[idx]
        inpSeq = []
        inpSeq_names=[]
        black=0
        tot=0
        back=0
        self.spatial_transform.randomize_parameters()

        for i in np.linspace(1, numFrame, self.seqLen, endpoint=False):
            tot+=1
            if self.frameType=='m':
              try:
                fl_name = vid_name + '/' + 'map' + str(int(np.floor(i))).zfill(4) + self.fmt
                img = Image.open(fl_name)
              except:
                try:
                  fl_name = vid_name + '/' + 'map' + str(int(np.floor(i+1))).zfill(4) + self.fmt
                  img = Image.open(fl_name)
                except:
                  try:
                    fl_name = vid_name + '/' + 'map' + str(int(np.floor(i-1))).zfill(4) + self.fmt
                    img = Image.open(fl_name)
                  except:
                    pass
              try:
                img=self.spatial_transform(img.convert('L'))
                inpSeq.append(img)
              except:
                if inpSeq:
                  inpSeq.append(inpSeq[-1])
                else:
                  inpSeq.append(torch.ones(size=(1,7,7)))

            if self.frameType=='r':
              fl_name = vid_name + '/' + 'rgb' + str(int(np.floor(i))).zfill(4) + self.fmt
              img = Image.open(fl_name)
              img=self.spatial_transform(img.convert('RGB'))
              fl_name_new=fl_name.replace('./GTEA61/processed_frames2/S1/','')
              #inpSeq.append(img)              
              inpSeq_names.append(fl_name_new)
        #inpSeq = torch.stack(inpSeq, 0)
        return inpSeq_names 